# Resultierende Implementierung

## Hier finden die Imports statt.

In [2]:
import math
import numpy as np
import random as rd
import pandas as pd
from scipy import stats 

import matplotlib.pyplot as plt
import matplotlib as mat
import holoviews as hv

import vaex as vaex
import vaex.jupyter.model as vjm

import seaborn as sns

import time

## Die Daten werden hier eingelesen.
###### Mit Übergabe als <class 'vaex.dataframe.DataFrameArrays'> mit Ausgabe der dafür benötigten Zeit

Hier wurde die Datei zwar über Pandas importiert, jedoch in Vaex übernommen, da die Vaex eigene Importierung zeitlich mehr benötigt und noch eine größere Nachbereitung als der Import von Pandas bräuchte.

In [3]:
start_proc = time.process_time()

datei = pd.read_csv(
    '/Users/samel/Documents/Hochschule/aktuelles Semester/03. Projekt zur Datenanalyse/Datein/annotation_example3.txt',
    sep=";",
    header=0
    )

datei = datei.rename(
    columns={'back_value                           ': 'backvalue'}
    )

df = vaex.from_pandas(datei)
#print(type(daten))

ende_proc = time.process_time()
print('Benötigte Systemzeit: {:5.3f}s'.format(ende_proc-start_proc))

df


Benötigte Systemzeit: 7.806s


#,pos_id,barcode,run_id,test_date,sign,value,backvalue
0,'1001200171739','100129017HD',10,'2017-07-07 03:07','s',88.33,131.05
1,'1001200171741','100129017HD',10,'2017-07-07 03:07','s',98.66,95.69
2,'1001200171742','100129017HD',10,'2017-07-07 03:07','s',35.03,70.9
3,'1001200171743','100129017HD',10,'2017-07-07 03:07','s',99.09,79.79
4,'1001200171744','100129017HD',10,'2017-07-07 03:07','s',82.09,143.5
...,...,...,...,...,...,...,...
"5,314,555",'1002000172921','100209017HD',9,'2017-07-07 09:07','s',88.9,128.91
"5,314,556",'1002401401231','100249140HD',9,'2017-07-07 09:07','s',93.52,91.56
"5,314,557",'1002401402813','100249140HD',9,'2017-07-07 09:07','s',88.53,93.61
"5,314,558",'1BCS600562415','1BCS6Y056LD',16,'2017-08-23 09:08','s',89.24,144.05


## Hier soll die Datei in Kategorien aufgeteilt werden
##### Nebenbei wird die Datei der Variable 'df_ohne_f' übergeben, die die Datei ohne die Werte der Kategorie bestehen und verweerden soll.

In [4]:
print('Länge des gesamten Datensatzes:', len(df), '\n', '\n')


f = df[df.sign == 'f'] # f => failure (ein Problem mit einem der Geräte)
print('Länge der Kategorie f:', len(f), '\n')#, f, '\n')

n = df[df.sign == 'n'] # n => neutral (nur Lösungsmittel, keine Substanz)
print('Länge der Kategorie n:', len(n), '\n')#, n, '\n')

p = df[df.sign == 'p'] # p => positiv (kontrolle mit -max- Effekt, bzw. verschiede Konzentrationen -> DWK)
print('Länge der Kategorie p:', len(p), '\n')#, p, '\n')

z = df[df.sign == 'z'] # z => zero (negative kontrolle, oder zweite kontrolle, ditto)
print('Länge der Kategorie z:', len(z), '\n')#, z, '\n')

s = df[df.sign == 's'] # s => substanz (getestetes kleines chem. Molekül)
print('Länge der Kategorie s:', len(s), '\n', '\n')#, s, '\n')


print("Wobei wir die Kategorie 'f' vernachläsigen werden, da es sich hierbei um Gerätefehler handelt:")
print( len(df), '-' ,len(df[df.sign == 'f']), ' =', len(s) + len(z) + len(p) + len(n))


Länge des gesamten Datensatzes: 5314560 
 

Länge der Kategorie f: 164522 

Länge der Kategorie n: 880350 

Länge der Kategorie p: 7744 

Länge der Kategorie z: 3872 

Länge der Kategorie s: 4258072 
 

Wobei wir die Kategorie 'f' vernachläsigen werden, da es sich hierbei um Gerätefehler handelt:
5314560 - 164522  = 5150038


Hier speichern wir alle Werte aus der importierten Datei, die nicht zur Kategorie 'f' angehören, in der Variable 'df_ohne_f'.

In [5]:
df_ohne_f = df[~(df['sign'] == 'f')]
print('\n', 'Länge von df:', len(df))
print('\n', 'Länge von df_ohne_f:', len(df_ohne_f))



 Länge von df: 5314560

 Länge von df_ohne_f: 5150038


## Inteervalle festlegen, um zukünftig Ausreißer entfernen zu können.
 
### Intervalsgrenzen bestimmen
Die Intervalle beziehen sich auf die jeweilige Kategorie (n, p, z, s).

In [6]:
print('Die Intervalle sind auf die jeweiligen signs bezogen und dem entsprechenden Feature, jedoch haben diese nicht den selben Intervall, was bei der Modelierung später angepasst werden muss:\n')

"""
f_stdValue = f.std('value')
f_stdBackvalue = f.std('backvalue')
f_meanValue = f.mean('value')
f_meanBackvalue = f.mean('backvalue')

f_ValueGrenzen = ( f_meanValue - (3 * f_stdValue),
                  f_meanValue + (3 * f_stdValue) )

f_BackvalueGrenzen = ( f_meanBackvalue - (3 * f_stdBackvalue),
                      f_meanBackvalue + (3 * f_stdBackvalue) )

print('f :', '\n', 'VALUE:', f_ValueGrenzen, 'BACK_VALUE:', f_BackvalueGrenzen, '\n')
"""


n_stdValue = n.std('value')
n_stdBackvalue = n.std('backvalue')
n_meanValue = n.mean('value')
n_meanBackvalue = n.mean('backvalue')


n_ValueGrenzen = ( n_meanValue - (3 * n_stdValue),
                  n_meanValue + (3 * n_stdValue) )

n_BackvalueGrenzen = ( n_meanBackvalue - (3 * n_stdBackvalue),
                      n_meanBackvalue + (3 * n_stdBackvalue) )

print('n:', '\n', 'VALUE:', n_ValueGrenzen, 'BACK_VALUE:', n_BackvalueGrenzen, '\n')



p_ValueGrenzen = ( p.mean('value') - (3 * p.std('value')),
                  p.mean('value') + (3 * p.std('value')) )

p_BackvalueGrenzen = ( p.mean('backvalue') - (3 * p.std('backvalue')),
                      p.mean('backvalue') + (3 * p.std('backvalue')) )

print('p:', '\n', 'VALUE:', p_ValueGrenzen, 'BACK_VALUE:', p_BackvalueGrenzen, '\n')



z_ValueGrenzen = ( z.mean('value') - (3 * z.std('value')),
                  z.mean('value') + (3 * z.std('value')) )

z_BackvalueGrenzen = ( z.mean('backvalue') - (3 * z.std('backvalue')),
                      z.mean('backvalue') + (3 * z.std('backvalue')) )

print('z:', '\n', 'VALUE:', z_ValueGrenzen, 'BACK_VALUE:', z_BackvalueGrenzen, '\n')



s_ValueGrenzen = ( s.mean('value') - (3 * s.std('value')),
                  s.mean('value') + (3 * s.std('value')) )

s_BackvalueGrenzen = ( s.mean('backvalue') - (3 * s.std('backvalue')),
                      s.mean('backvalue') + (3 * s.std('backvalue')) )

print('s:', '\n', 'VALUE:', s_ValueGrenzen, 'BACK_VALUE:', s_BackvalueGrenzen, '\n')


Die Intervalle sind auf die jeweiligen signs bezogen und dem entsprechenden Feature, jedoch haben diese nicht den selben Intervall, was bei der Modelierung später angepasst werden muss:

n: 
 VALUE: (67.79281497978246, 132.7155888368814) BACK_VALUE: (-86.29620736649818, 308.5234996934133) 

p: 
 VALUE: (-89.2820048963031, 191.3068499376251) BACK_VALUE: (-10235.072866094875, 16227.178395537014) 

z: 
 VALUE: (-86.97366500628134, 204.09270942776874) BACK_VALUE: (-236.47778804234548, 435.3835421745769) 

s: 
 VALUE: (34.63217727631585, 151.129811708368) BACK_VALUE: (-410.9874897252928, 602.2923863216954) 



Das hier berechnete Intervall ist im Bezug auf die gesamte Datei ohne die Kategorie 'f', wobei sich hier nicht auf das jeweilige sign bezogen wird und somit problematisch sein könnte, aber zur Modllierung im späteren Verlauf besser zu verwenden ist und somit zukünftik damit gearbeitet wird.
Die vorherigen Intervallsgrenzen wurden zur Veranschaunlichung erstellt, um einen Vergleich bzw. Abweichung besser modelieren zu können.

In [7]:
df_ValueGrenzen = ( df.mean('value') - (3 * df.std('value')),
                   df.mean('value') + (3 * df.std('value')))

df_BackvalueGrenzen = ( df.mean('backvalue') - (3 * df.std('backvalue')),
                       df.mean('backvalue') + (3 * df.std('backvalue')) )

print('Wenn wir die Intevalle der importierten Datei bestimmen, was in df gespeichert wurde, kommt folgendes dabei raus:')
print('    df:', '\n    VALUE:', df_ValueGrenzen, 'BACK_VALUE:', df_BackvalueGrenzen, '\n')
print("    Mittelwert von 'value':", df.mean('value'), "\n    Mittelwert von 'backvalue':", df.mean('backvalue'), '\n\n')



df_ohne_f_ValueGrenzen = ( df_ohne_f.mean('value') - (3 * df_ohne_f.std('value')),
                          df_ohne_f.mean('value') + (3 * df_ohne_f.std('value')))

df_ohne_f_BackvalueGrenzen = ( df_ohne_f.mean('backvalue') - (3 * df_ohne_f.std('backvalue')),
                              df_ohne_f.mean('backvalue') + (3 * df_ohne_f.std('backvalue')) )

print('Wir verwenden vorhin beschrieben, jedoch die Intevalle der Datei ohne der Kategorie 'f', was in df_ohne_f gespeichert wurde und kommt folgendes dabei raus:')
print('    df_ohne_f:', '\n    VALUE:', df_ohne_f_ValueGrenzen, 'BACK_VALUE:', df_ohne_f_BackvalueGrenzen, '\n')
print("    Mittelwert von 'value':", df_ohne_f.mean('value'), "\n    Mittelwert von 'backvalue':", df_ohne_f.mean('backvalue'))


Wenn wir die Intevalle der importierten Datei bestimmen, was in df gespeichert wurde, kommt folgendes dabei raus:
    df: 
    VALUE: (38.07067313089432, 150.27277827053746) BACK_VALUE: (-713.5645279374443, 923.3391621498805) 

    Mittelwert von 'value': 94.17172570071568 
    Mittelwert von 'backvalue': 104.88731710621813 


Wir verwenden vorhin beschrieben, jedoch die Intevalle der Datei ohne der Kategorie , was in df_ohne_f gespeichert wurde und kommt folgendes dabei raus:
    df_ohne_f: 
    VALUE: (38.07129460869005, 150.03393152750283) BACK_VALUE: (-669.3395979545287, 874.6586361363945) 

    Mittelwert von 'value': 94.05261306809646 
    Mittelwert von 'backvalue': 102.65951909093293


=> Daraus ist erkenntlich, dass für den 'value' ein geringer Unterschied besteht ob der sign 'f' mit betrachtet wird oder nicht, wo hingegen es beim 'backvalue' es einen größeren Unterschied macht. Es werden zwar wenn wir die einzelnen sign's nochmal betrachten, einen Unterschied machen, ob wir sie zusammenhängend oder einzelnd betrachten, indem besipielsweise von den sign's n, z und s mehr Werte mit übernommen und betrachtet, wo es beim p so ist, dass es weniger werte sind, die betrachtet werden.

Um es genauer Auszudrücken: Das betrachten der Intervallsgrenzen der Datei ohne der Kateegorie 'f' sorgt dafür, dass wir 4995633 Zeilen von den gesamten 5314560 haben, wenn wir jedoch die Kategorien einzeln betrachtet und derren Zeilen zusammengefügt hätten, wären es stattdessen 4992498, was eine Abweichung von 3135 Zeilen ist.

In [9]:
df_cleaned_without_f = df_ohne_f[(df_ohne_f['value'] <= df_ohne_f_ValueGrenzen[1] )
                                 & (df_ohne_f['value'] >= df_ohne_f_ValueGrenzen[0] )
                                 & (df_ohne_f['backvalue'] <= df_ohne_f_BackvalueGrenzen[1] )
                                 & (df_ohne_f['backvalue'] >= df_ohne_f_BackvalueGrenzen[0] )
                                ]

print('  df_without_f:', len(df_ohne_f), '=> df_cleaned_without_f:', len(df_cleaned_without_f))#, '\n    -> Der Wert unterscheidet sich um', len(df_cleaned_without_f)-gesamt_anz, 'im verhältis zu der Menge, wo die signs einzeln betrachtet wurden.', '\n')



  df_without_f: 5150038 => df_cleaned_without_f: 4995633
